In [13]:
# Unsloth 라이브러리에서 FastLanguageModel을 임포트
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # 최대 시퀀스 길이를 설정 ( 텍스트의 최대 길이를 지정)
dtype = None  # 자동 감지를 위해 None 설정. Tesla T4는 Float16, Ampere+는 Bfloat16 사용. 모델의 파라미터를 저장할 데이터 타입
load_in_4bit = True  # 메모리 사용량을 줄이기 위해 4비트 양자화 사용. 다만 양자화에 따른 손실이 있어서 필요에 따라 False로 설정 가능

# 사전 학습된 모델과 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",  # 사용할 모델 이름
    max_seq_length = max_seq_length,         # 설정한 최대 시퀀스 길이
    dtype = dtype,                           # 데이터 타입 설정
    load_in_4bit = load_in_4bit,             # 4비트 양자화 여부
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2+cu118. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [14]:
# PEFT(파라미터 효율적 파인튜닝) 모델 설정
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA 랭크 설정. 8, 16, 32, 64, 128 권장. r 값이 클수록 모델이 더 많은 정보를 학습할 수 있지만, 너무 크면 메모리를 많이 사용
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],  # PEFT 적용할 모듈 목록. 모델의 특정 부분(모듈)에만 학습
    lora_alpha = 16,        # LoRA 알파 설정 LoRA라는 기술이 얼마나 강하게 작용할지 조절
    lora_dropout = 0,       # LoRA 드롭아웃 설정. 0으로 최적화
    bias = "none",          # 바이어스 설정. "none"으로 최적화

    # "unsloth" 사용 시 VRAM 절약 및 배치 사이즈 2배 증가
    # 학습할 때 메모리를 절약하는 방법을 사용하는 설정
    use_gradient_checkpointing = "unsloth",  # 매우 긴 컨텍스트를 위해 "unsloth" 설정
    random_state = 3407,    # 랜덤 시드 설정
    use_rslora = False,     # 랭크 안정화 LoRA 사용 여부
    loftq_config = None,    # LoftQ 설정 (사용하지 않음)
)

In [17]:
# 모델에게 주어질 텍스트의 형식을 정의
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

# 맨윗줄: 모델에게 앞으로 제공될 지시사항을 기반으로 적절한 응답을 작성하라고 말함.
# 모델이 어떤 작업을 수행해야 하는지 명확히 이해할 수 있도록 도와줌
# Instruction:은 지시사항이 제공될 부분
# Response:는 모델이 생성해야 할 응답이 제공될 부분

# EOS 토큰 가져오기 (생성 종료를 위해 필요)
EOS_TOKEN = tokenizer.eos_token  # 반드시 EOS_TOKEN을 추가해야 함
EOS_TOKEN

'<|end_of_text|>'

In [18]:
# 프롬프트 포맷팅 함수 정의
def formatting_prompts_func(examples):
    instructions = examples["유저"]  # 데이터셋의 'instruction' 필드
    outputs      = examples["챗봇"]       # 데이터셋의 'output' 필드
    texts = []
    for instruction, output in zip(instructions, outputs):
                                                           # EOS_TOKEN을 추가하지 않으면 생성이 무한히 계속됨
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN  # 프롬프트 형식에 맞게 텍스트 생성
        texts.append(text)
    return { "text" : texts, }  # 'text' 필드로 반환

#from datasets import load_dataset  # Hugging Face datasets 라이브러리 임포트

# 데이터셋 로드 (Teddy Lee의 QA 데이터셋 미니 버전)
#dataset = load_dataset("teddylee777/QA-Dataset-mini", split = "train")

# 프롬프트 포맷팅 함수 적용하여 데이터셋 변환
#dataset = dataset.map(formatting_prompts_func, batched = True,)

from datasets import load_dataset
dataset = load_dataset( "csv", data_files = "/home/alpaco/chat_bot/wellnis.csv", split = "train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# 예제 엑셀 형태로 데이터셋을 만든다면?
# from datasets import load_dataset
# dataset = load_dataset( "csv", data_files = "data.csv", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched=True)
#
#

In [19]:
dataset['text'][:5]

['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n다 제 잘못이에요.\n\n### Response:\n낚시를 하듯이 조금 더 먼 곳을 보고, 조금 더 마음을 비우면 편안해질 거예요.<|end_of_text|>',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n어렸을 때부터 엄마보다는 아빠랑 더 친했어요.\n\n### Response:\n그러시군요. 아버지에 대해 더 들려주세요.<|end_of_text|>',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n몸무게도 늘고요.\n\n### Response:\n체중이 단기간에 많이 늘어나면 건강에 좋지 않다고 해요.<|end_of_text|>',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n눈물이 날 거 같더라고.\n\n### Response:\n감수성이 풍부하다는 증거 같아요. 눈물이 많은 사람 중에 나쁜 사람은 없잖아요.<|end_of_text|>',
 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n회사를 옮기면서

In [20]:
## 학습 설정
from trl import SFTTrainer  # TRL 라이브러리에서 SFTTrainer 임포트
from transformers import TrainingArguments  # 트랜스포머 라이브러리에서 TrainingArguments 임포트
from unsloth import is_bfloat16_supported  # BFloat16 지원 여부 확인 함수 임포트

# SFTTrainer 인스턴스 생성
trainer = SFTTrainer(
    model = model,                           # 학습할 모델
    tokenizer = tokenizer,                   # 사용할 토크나이저
    train_dataset = dataset,                 # 학습할 데이터셋 ★★★★★★★★
    dataset_text_field = "text",             # 데이터셋의 텍스트 필드 이름 ★★★★★★★★
    max_seq_length = max_seq_length,         # 최대 시퀀스 길이
    dataset_num_proc = 2,                    # 데이터셋 전처리에 사용할 프로세스 수 cpu
    packing = False,                         # 짧은 시퀀스의 경우 packing을 비활성화 (학습 속도 5배 향상 가능)
    args = TrainingArguments(
        per_device_train_batch_size = 2,     # 디바이스 당 배치 사이즈
        gradient_accumulation_steps = 4,     # 그래디언트 누적 단계 수
        warmup_steps = 5,                     # 워밍업 스텝 수
        # num_train_epochs = 1,               # 전체 학습 에폭 수 설정 가능
        max_steps = 60,                       # 최대 학습 스텝 수
        learning_rate = 2e-4,                 # 학습률
        fp16 = not is_bfloat16_supported(),   # BFloat16 지원 여부에 따라 FP16 사용
        bf16 = is_bfloat16_supported(),       # BFloat16 사용 여부
        logging_steps = 1,                    # 로깅 빈도
        optim = "adamw_8bit",                  # 옵티마이저 설정 (8비트 AdamW)
        weight_decay = 0.01,                  # 가중치 감쇠
        lr_scheduler_type = "linear",         # 학습률 스케줄러 타입
        seed = 3407,                           # 랜덤 시드 설정
        output_dir = "outputs",                # 출력 디렉토리
    ),
)

## 학습 실행
trainer_stats = trainer.train()  # 모델 학습 시작 # 3d6e4b9a0ddcd9edfafad59dcde0fd8c666954fd

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 857 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,3.274200
2,3.394200
3,3.408800
4,3.288600
5,3.069200
6,2.557600
7,2.456700
8,2.317200
9,2.000000
10,1.810600


In [21]:
# 모델 저장 로컬폴더에다가 저장하는 방식
model.save_pretrained("wellnis")  # Local saving
tokenizer.save_pretrained("wellnis")

# 이 이외에 허깅페이스나 다른 hub에 push해서 저장하는 방법이 있음
# 다만, 업로드 속도와 다운로드 속도를 고려해야함.

('wellnis/tokenizer_config.json',
 'wellnis/special_tokens_map.json',
 'wellnis/tokenizer.json')

In [22]:
from unsloth import FastLanguageModel
import torch

# 저장된 경로 지정
save_directory = "/home/alpaco/chat_bot/wellnis"

# 모델과 토크나이저 불러오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_directory,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,  # 양자화 옵션을 동일하게 설정
)


==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2+cu118. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [23]:
# ...기존 모델/토크나이저 로딩 코드 이후...

from transformers import TextStreamer

# 1. 원샷 프롬프트 정의
one_shot_prompt = """
### Instruction:
너무 무기력해.

### Response:
증상: 무기력증 무기력감은 삶의 목적과 의미를 잃고, 아무 일도 하지 못하는 상태를 말합니다. 이 증상은 우울증과 함께 나타날 수 있지만, 우울감만으로는 설명이 안되는 경우가 많습니다. 무기력감은 수면 문제, 식욕 변화, 집중력 저하 등의 증상이 동반될 수 있습니다.

### Instruction:
너무 우울해.

### Response:
"""

In [24]:
# 2. 추론을 위한 입력 준비
inputs = tokenizer(
    one_shot_prompt,
    return_tensors="pt"
).to("cuda")

# 3. TextStreamer 초기화
text_streamer = TextStreamer(tokenizer)

# 4. 모델을 사용하여 텍스트 생성 및 스트리밍 출력
output = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

<|begin_of_text|>
### Instruction:
너무 무기력해.

### Response:
증상: 무기력증 무기력감은 삶의 목적과 의미를 잃고, 아무 일도 하지 못하는 상태를 말합니다. 이 증상은 우울증과 함께 나타날 수 있지만, 우울감만으로는 설명이 안되는 경우가 많습니다. 무기력감은 수면 문제, 식욕 변화, 집중력 저하 등의 증상이 동반될 수 있습니다.

### Instruction:
너무 우울해.

### Response:
증상: 우울감 우울감은 우울한 기분이 지속적으로 지속되는 것을 말합니다. 우울감은 신체적 증상(의욕 상실, 식욕 변화, 수면 장애, 피로감 등)이 동반될 수 있습니다.<|end_of_text|>


In [27]:
# 5. 답변만 추출
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text.split("### Response:")[-1].strip()

'증상: 우울감 우울감은 우울한 기분이 지속적으로 지속되는 것을 말합니다. 우울감은 신체적 증상(의욕 상실, 식욕 변화, 수면 장애, 피로감 등)이 동반될 수 있습니다.'